In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt
import math, os, re, time, random, string
import numpy as np, pandas as pd, seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
from collections import defaultdict
import wordcloud
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from tensorflow.keras.layers import *
from tensorflow.keras import Input, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from transformers import TFAutoModel, AutoTokenizer, BertTokenizer, TFBertModel

from tokenizers import BertWordPieceTokenizer

In [ ]:
test = pd.read_csv('/kaggle/input/rscience-popular-comment-removal/reddit_test.csv', encoding = 'latin-1')
df1 = pd.read_csv('/kaggle/input/rscience-popular-comment-removal/reddit_train.csv', encoding = 'latin-1')
train, val = train_test_split(df1, test_size=0.7)
y_train = train['REMOVED']
y_val = val['REMOVED']
y_test = test['REMOVED']

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square bracketsremove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

stop_words = stopwords.words('english')+['u', 'im', 'c']

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text 

stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

def preprocess(df):
    df = df.apply(clean_text)
    df = df.apply(remove_stopwords)
    df = df.apply(stemm_text)
    df = df.dropna()
    return df

train['BODY'] = preprocess(train['BODY'])
val['BODY'] = preprocess(val['BODY'])
test['BODY'] = preprocess(test['BODY'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def bert_encode(data,maximum_len) :
    input_ids = []
    attention_masks = []
  

    for i in data:
        encoded = tokenizer.encode_plus(i,
                                        truncation=True,
                                        max_length=maximum_len,
                                        pad_to_max_length=True,
                                        return_attention_mask=True)
      
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

train_input_ids, train_attention_masks = bert_encode(train['BODY'], 200)
val_input_ids, val_attention_masks = bert_encode(val['BODY'], 200)
test_input_ids, test_attention_masks = bert_encode(test['BODY'], 200)

In [ ]:
def create_model(bert_model):
    
    input_ids = tf.keras.Input(shape=(200,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(200,),dtype='int32')    
    
    transformer_layer = bert_model([input_ids,attention_masks])
    output = transformer_layer[0]

 #   output = tf.keras.layers.Dropout(0.2)(output)
  #  output = tf.keras.layers.Dense(32,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
bert_model = TFAutoModel.from_pretrained('distilbert-base-uncased', num_labels=2)


In [ ]:
model = create_model(bert_model)
model.summary()

history = model.fit(
    [train_input_ids, train_attention_masks],
    y_train,
    validation_data = ([val_input_ids, val_attention_masks], y_val), 
    epochs=10)